# Debugging YAML

### Introduction

One of the trickier components of DBT is simply making sure that we have properly set up the YAML syntax.  For that reason, we'll talk through some debugging techniques when working with YAML in DBT.

### Debugging Yaml

Before moving on, it's worth making sure we understand some of the yaml syntax.  Let's take another look at the syntax that we used to add a test to our models.

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns:
      - name: first_name
      tests:
          - not_null
      - name: last_name
        tests:
          - not_null
```

Starting with `models`, notice that we then go two spaces in on the next line.  This is to indicate that we are about to describe a kind of model, `dim_films`.  We start the next line with a `-`, because there can be a list of models, and the `-` is used for each element of a list.

Getting the syntax correct, can be pretty tricky, but luckily DBT will give us some hints if we get it wrong.  For example, let's purposely make a mistake by not indenting the `columns` key under name.

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns:
    - name: first_name
      tests:
          - not_null
      - name: last_name
        tests:
          - not_null
```

Then run `dbt test`.

```bash
Parsing Error
  Error reading northwinds_dbt: staging/hubspot/schema.yaml - Runtime Error
    Syntax error near line 7
    ------------------------------
    4  |   - name: stg_hubspot_contacts
    5  |     columns:
    6  |       - name: first_name
    7  |       tests:
    8  |           - not_null
    9  |           - accepted_values:
    10 |               values: ['fran', 'daphne']

    Raw Error:
    ------------------------------
    while parsing a block collection
      in "<unicode string>", line 6, column 7
    did not find expected '-' indicator
      in "<unicode string>", line 7, column 7
```

Here, our tests are not  run because DBT is unable to interpret our yaml.  Notice that dbt is indicating an error, and also guessing the line number of 7x.  Where our line number of 7 is the `tests` key.  And it's pretty close!  The real error is of course right above, in line 6.

If you're unable to spot which error is the problem, it's best to remove some of the yaml just to get it working.  For example, the next step might be to update the yaml to the following:

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
```

Then we can run `dbt test` again to see if our yaml compiles.  If it does, we can keep adding onto our code, by this time specifying the `columns`.  

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns: 
```

```bash
Parsing Error
  Invalid models config given in models/staging/hubspot/schema.yaml @ models: {'name': 'stg_hubspot_contacts', 'columns': None, 'original_file_path': 'models/staging/hubspot/schema.yaml', 'yaml_key': 'models', 'package_name': 'northwinds_dbt'} - at path ['columns']: None is not of type 'array'
```

So this time, DBT is telling us that there are `Invalid models config given`.  And if we look at the bottom, it says that under columns, `None is not a type of array`.  Let's add back in our `first_name` column.

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns: 
      - name: first_name
      
```

And now, our code will at least compile.  And to get it to test something, we add back in the corresponding `test`.

Ok, so now we have fixed our YAML and we can see that it properly compiles.  

Finally, add back in the original test.

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns:
      - name: first_name
      tests:
          - not_null
      - name: last_name
        tests:
          - not_null
```

### Resources

[DBT Test Documentation](https://docs.getdbt.com/reference/resource-properties/tests)